## **Data Cleaning**

### Imports

In [141]:
pip install scikit-learn==1.5.0


In [142]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from google.colab import drive
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import joblib as jb
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading the data

In [143]:
df = pd.read_csv("drive/My Drive/male_players (legacy).csv", chunksize = 1000)
Df = pd.DataFrame()
for chunk in df:
    Df = pd.concat([Df,chunk]).reset_index(drop = True)

In [144]:
Df

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


### Dropping columns with greater that 30% of its values being null

In [145]:
L = []
L_less = []
for i in Df.columns:
    if((Df[i].isnull().sum())<(0.4*(Df.shape[0]))):
        L.append(i)
    else:
        L_less.append(i)
Df = Df[L]

In [146]:
Df

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [147]:
numeric_data = Df.select_dtypes(include=[np.number])
categorical_data = Df.select_dtypes(exclude=[np.number])

In [148]:
numeric_data.columns

Index(['player_id', 'fifa_version', 'fifa_update', 'overall', 'potential',
       'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'league_id',
       'league_level', 'club_team_id', 'club_jersey_number',
       'club_contract_valid_until_year', 'nationality_id', 'weak_foot',
       'skill_moves', 'international_reputation', 'release_clause_eur', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'me

In [149]:
num_corr = numeric_data.corr()

In [150]:
type(num_corr)

pandas.core.frame.DataFrame

In [151]:
num_corr["overall"].sort_values(ascending=False)

overall                           1.000000
movement_reactions                0.845753
potential                         0.695362
mentality_composure               0.683307
passing                           0.681114
wage_eur                          0.611368
dribbling                         0.606575
release_clause_eur                0.578914
value_eur                         0.566827
physic                            0.513279
attacking_short_passing           0.499757
mentality_vision                  0.493282
international_reputation          0.485234
skill_long_passing                0.483808
shooting                          0.482498
power_shot_power                  0.480297
age                               0.458390
skill_ball_control                0.457011
skill_curve                       0.415863
power_long_shots                  0.409086
mentality_aggression              0.398310
attacking_crossing                0.396833
skill_fk_accuracy                 0.387269
attacking_v

In [152]:
for x in num_corr["overall"].index:
    if not(num_corr["overall"][x]>0.3 or num_corr["overall"][x]<-0.3):
        numeric_data.drop([x], axis = 1, inplace = True)

In [153]:
numeric_data.columns

Index(['player_id', 'overall', 'potential', 'value_eur', 'wage_eur', 'age',
       'skill_moves', 'international_reputation', 'release_clause_eur',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_reactions', 'power_shot_power', 'power_stamina',
       'power_strength', 'power_long_shots', 'mentality_aggression',
       'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
       'mentality_penalties', 'mentality_composure'],
      dtype='object')

In [154]:
num_arr = ['player_id','release_clause_eur']
numeric_data.drop(columns=num_arr, inplace= True)

In [155]:
categorical_data

,player_url,fifa_update_date,short_name,long_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,1987-06-24,La Liga,FC Barcelona,CF,2004-07-01,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",1985-02-05,La Liga,Real Madrid CF,LW,2009-07-01,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",1984-01-23,Bundesliga,FC Bayern München,SUB,2009-08-28,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,1981-10-03,Ligue 1,Paris Saint-Germain,ST,2012-07-01,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,1986-03-27,Bundesliga,FC Bayern München,GK,2011-07-01,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,/player/269011/ziye-zhao/230002,2022-09-26,Zhao Ziye,赵梓业,RW,2003-10-01,Super League,Hebei CFFC,RES,2022-05-01,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,/player/269019/junjie-wu/230002,2022-09-26,Wu Junjie,吴俊杰,LB,2003-06-02,Super League,Guangzhou R&F,RES,2022-05-01,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,/player/271093/kailin-barlow/230002,2022-09-26,K. Barlow,Kailin Barlow,CDM,2003-06-28,Premier Division,Sligo Rovers,RES,2021-11-02,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,/player/271555/gideon-tetteh/230002,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",2005-05-20,Premier Division,Shamrock Rovers,RES,2022-06-24,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [156]:
arr = ['player_url','fifa_update_date','player_positions', 'short_name', 'long_name','dob', 'league_name', 'club_name', 'club_position','club_joined_date','nationality_name','real_face', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam',
       'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm','rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', "ls", "st", "rs"]
categorical_data.drop(columns=arr, inplace= True)

In [157]:
categorical_data.columns

Index(['preferred_foot', 'work_rate', 'body_type'], dtype='object')

In [158]:
im = IterativeImputer(max_iter = 10, random_state = 0 )
imputed_numeric = im.fit_transform(numeric_data)
numeric_data = pd.DataFrame(imputed_numeric,columns = numeric_data.columns)

In [159]:
si = SimpleImputer(strategy = "most_frequent")
imputed_categorical = si.fit_transform(categorical_data)
categorical_data = pd.DataFrame(imputed_categorical,columns = categorical_data.columns)

In [160]:
categorical_data.columns

Index(['preferred_foot', 'work_rate', 'body_type'], dtype='object')

In [161]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)

In [162]:
for x in categorical_data.columns:
     categorical_data[x] = label_encoder.fit_transform(categorical_data[x])
categorical_data

,preferred_foot,work_rate,body_type
0,0,7,3
1,1,1,5
2,0,1,4
3,1,7,5
4,1,8,5
...,...,...,...
161578,1,8,1
161579,0,8,5
161580,1,2,4
161581,1,8,4


In [163]:
onehot_encoded = onehot_encoder.fit_transform(categorical_data)
feature_names = onehot_encoder.get_feature_names_out(categorical_data.columns)
print(len(feature_names))


21


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [164]:
onehot_encoded_categorical = pd.DataFrame(onehot_encoded, columns = feature_names)

In [165]:
Df = pd.concat([numeric_data,onehot_encoded_categorical], axis = 1).reset_index(drop = True)

In [166]:
len(Df.columns)

54

### Data Training, Evaluation and Testing

In [167]:
y = Df["overall"]
x = Df.drop("overall", axis=1)

In [168]:
scale=StandardScaler()
X=scale.fit_transform(x)

In [169]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [170]:
xgb_regressor = XGBRegressor()
xgb_regressor.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [171]:
feature_importances = xgb_regressor.feature_importances_

In [172]:
type(feature_importances)

numpy.ndarray

In [173]:
feature_importance_df = pd.DataFrame({
    'Feature': x.columns,
    'Importance': feature_importances
})

In [174]:
feature_importance_df.sort_values("Importance",ascending=False)


,Feature,Importance
1,value_eur,0.600832
21,movement_reactions,0.113827
3,age,0.061725
2,wage_eur,0.045750
0,potential,0.041745
5,international_reputation,0.037628
9,defending,0.015775
4,skill_moves,0.012896
20,skill_ball_control,0.008416
8,dribbling,0.008016


In [175]:
feature_importance_df.shape

(53, 2)

In [176]:
arr = []
for x in feature_importance_df.iloc[:10,0]:
    arr.append(x)

In [177]:
Df = Df[arr]

In [178]:

x = Df

In [179]:
x


,potential,value_eur,wage_eur,age,skill_moves,international_reputation,shooting,passing,dribbling,defending
0,95.0,100500000.0,550000.0,27.0,4.0,5.0,89.000000,86.000000,96.00000,27.00000
1,92.0,79000000.0,375000.0,29.0,5.0,5.0,93.000000,81.000000,91.00000,32.00000
2,90.0,54500000.0,275000.0,30.0,4.0,5.0,86.000000,83.000000,92.00000,32.00000
3,90.0,52500000.0,275000.0,32.0,4.0,5.0,91.000000,81.000000,86.00000,34.00000
4,90.0,63500000.0,300000.0,28.0,1.0,5.0,29.075512,32.431673,33.95528,38.80932
...,...,...,...,...,...,...,...,...,...,...
161578,61.0,110000.0,700.0,18.0,2.0,1.0,48.000000,40.000000,47.00000,24.00000
161579,58.0,110000.0,750.0,19.0,2.0,1.0,25.000000,29.000000,34.00000,48.00000
161580,58.0,110000.0,500.0,19.0,2.0,1.0,36.000000,43.000000,46.00000,43.00000
161581,70.0,150000.0,500.0,17.0,2.0,1.0,50.000000,36.000000,46.00000,20.00000


In [180]:
scale=StandardScaler()
X=scale.fit_transform(x)

In [181]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size = 0.2, random_state = 42)

###Training Individual Models

In [182]:
cv=KFold(n_splits=5)
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoost': GradientBoostingRegressor(random_state=42)
}
mae_scorer = make_scorer(mean_absolute_error)

In [183]:
for name,model in models.items():
  scores = cross_val_score(model, Xtrain, Ytrain, cv=cv, scoring=mae_scorer)
  print(f'{name} MAE: {scores.mean():.4f} (+/- {scores.std():.4f})')

RandomForest MAE: 0.4679 (+/- 0.0037)
XGBoost MAE: 0.6237 (+/- 0.0046)
GradientBoost MAE: 0.9125 (+/- 0.0031)


In [184]:
Df

,potential,value_eur,wage_eur,age,skill_moves,international_reputation,shooting,passing,dribbling,defending
0,95.0,100500000.0,550000.0,27.0,4.0,5.0,89.000000,86.000000,96.00000,27.00000
1,92.0,79000000.0,375000.0,29.0,5.0,5.0,93.000000,81.000000,91.00000,32.00000
2,90.0,54500000.0,275000.0,30.0,4.0,5.0,86.000000,83.000000,92.00000,32.00000
3,90.0,52500000.0,275000.0,32.0,4.0,5.0,91.000000,81.000000,86.00000,34.00000
4,90.0,63500000.0,300000.0,28.0,1.0,5.0,29.075512,32.431673,33.95528,38.80932
...,...,...,...,...,...,...,...,...,...,...
161578,61.0,110000.0,700.0,18.0,2.0,1.0,48.000000,40.000000,47.00000,24.00000
161579,58.0,110000.0,750.0,19.0,2.0,1.0,25.000000,29.000000,34.00000,48.00000
161580,58.0,110000.0,500.0,19.0,2.0,1.0,36.000000,43.000000,46.00000,43.00000
161581,70.0,150000.0,500.0,17.0,2.0,1.0,50.000000,36.000000,46.00000,20.00000


## Ensemble methods
### RandomForest, XGBoost, Gradient Boost Regressors


In [ ]:
#Random Forest
cv=KFold(n_splits=5)
rf = RandomForestRegressor()
PARAMETERS_rf ={
"n_estimators":[10,50, 100, 200]
}
model_rf=GridSearchCV(rf,param_grid=PARAMETERS_rf,cv=cv,scoring="neg_mean_squared_error")
model_rf.fit(Xtrain, Ytrain)
jb.dump(model_rf, open('drive/My Drive/' + rf.__class__.__name__ + '.joblib', 'wb'))
y_pred = model_rf.predict(Xtest)
print(
    f"""
    Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
    Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
    R2 Score = {r2_score(y_pred,Ytest)}
    """
    )




In [ ]:
#GradientBoostRegressor
cv=KFold(n_splits=3)
gb = GradientBoostingRegressor(n_iter_no_change=10, validation_fraction=0.1)
PARAMETERS_gb ={
"max_depth":[3,5,8],
"min_samples_leaf":[1,5,10],
"min_samples_split":[2,5],
# "min_child_weight":[1,5,15],​
"learning_rate":[0.5, 0.1],
"n_estimators":[100]}
model_gs=GridSearchCV(gb,param_grid=PARAMETERS_gb,cv=cv,scoring="neg_mean_squared_error", n_jobs=-1)
model_gs.fit(Xtrain, Ytrain)
jb.dump(model_gs, open('drive/My Drive/' + gb.__class__.__name__ + '.joblib', 'wb'))
y_pred = model_gs.predict(Xtest)
print(model_gs.__class__.__name__,
    f"""
    Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
    Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
    Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
    R2 Score = {r2_score(y_pred,Ytest)}
    """)


In [ ]:
#XGBOOST
cv=KFold(n_splits=3)
xg = XGBRegressor()
PARAMETERS_xg ={
"n_estimators":[100,500,1000]}
model_xg=GridSearchCV(xg,param_grid=PARAMETERS_xg,cv=cv,scoring="neg_mean_squared_error")
model_xg.fit(Xtrain, Ytrain)
best_model_xg = model_xg.best_estimator_
print("Best parameters found: ", model_xg.best_params_)

jb.dump(model_xg, open('drive/My Drive/' + xg.__class__.__name__ + '.joblib', 'wb'))
y_pred = model_xg.predict(Xtest)

print(xg.__class__.__name__,
    f"""
    Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
    Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
    R2 Score = {r2_score(y_pred,Ytest)}
    """)

In [ ]:
df = pd.read_csv("drive/My Drive/players_22.csv", chunksize = 1000)
new_Df = pd.DataFrame()
for chunk in df:
    new_Df = pd.concat([new_Df,chunk]).reset_index(drop = True)

In [ ]:
y = new_Df['overall']
X = new_Df[arr]

In [ ]:
X

In [ ]:
imputed_numeric = im.fit_transform(X)
X = pd.DataFrame(imputed_numeric,columns = X.columns)
X=scale.fit_transform(X)

In [ ]:
y_pred = model_rf.predict(X)
print(model_rf.__class__.__name__,
    f"""
    Mean Absolute Error = {mean_absolute_error(y_pred,y)},
    Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y))},
    R2 Score = {r2_score(y_pred,y)}
    """)

In [ ]:
type(Xtrain)

In [ ]:
sklearn.__version__